In [1]:
import json

In [10]:
import json
ENTITIES_FILEPATH='camera_trainingset/camera-ENTITIES.json'
JSON_FILES_FILEPATH = 'camera_trainingset/camera-JSON_FILES.json'
with open(ENTITIES_FILEPATH) as json_file:
    matching = json.load(json_file)
with open(JSON_FILES_FILEPATH) as json_file:
    json_filepaths = json.load(json_file)

In [11]:
len(json_filepaths)

29787

In [28]:
import os

filepaths = {}
DATASET_FILEPATH = 'camera_dataset/2013_camera_dataset/'
for filepath in json_filepaths:
    json_id = filepath['resource_id']
    domain = filepath['source_name']
    filename = str(filepath['json_number'])+'.json'
    filepaths[json_id]= os.path.join(DATASET_FILEPATH,domain,filename)

In [37]:
# positives samples è una tupla record1,record2,matching/nomatching
positives_samples = []
for entity in matching:
    instances = entity['instances']
    i=0
    for instance in instances:
        filepath = filepaths[instance]
        with open(filepath) as json_file:
            record = json.load(json_file)
            left_page_title = record['<page title>']
        for j in range(i+1,len(instances)):
            filepath = filepaths[instances[j]]
            with open(filepath) as json_file:
                record = json.load(json_file)
                right_page_title = record['<page title>']
                positives_samples.append((left_page_title,right_page_title,1))

In [46]:
positives_samples[200]

('Nikon D3200 DSLR Camera Import 18 55 VR w A and Tele 7 Lens 24GB Bundle | eBay',
 'Nikon D3200 DSLR Camera Rs.24850 Price in India - Buy Nikon D3200 DSLR Camera Black Online - Nikon : Flipkart.com',
 1)

In [65]:
all_sources = os.listdir('camera_dataset/2013_camera_dataset')

In [66]:
import random
random.choice(all_sources)

'www.garricks.com.au'

In [57]:
import numpy as np
GLOVE_DIR='glove'
GLOVE_FILENAME='glove.840B.300d.txt'
wordToEmbeddingMap = {}
with open(GLOVE_DIR+'/'+GLOVE_FILENAME) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, 'f', sep=' ')
    # we found out that glove.840B.300d.txt is supposed to contain 301-item lines (the token and its 300 weights) but some lines do contain 
    # more than one str tokens, so we ignore those malformed lines (whose corresponding coefs variable is an empty array)
    if len(coefs) != 0:
        wordToEmbeddingMap[word] = coefs

In [119]:
import nltk
stopwords = nltk.download("stopwords")

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}','|','-'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vincenzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
BASE_DIR ='camera_dataset/2013_camera_dataset/'
source1 = random.choice(all_sources)
source2 = random.choice(all_sources)
while(source2==source1):
    source2 = random.choice(all_sources)
all_files_source1 = os.listdir(BASE_DIR+source1)
all_files_source2 = os.listdir(BASE_DIR+source2)

In [91]:
all_json_source1 = list(filter(lambda s: s.endswith('.json'), all_files_source1))
all_json_source2 = list(filter(lambda s: s.endswith('.json'),all_files_source2))

In [92]:
len(all_files_source1)

1480

In [93]:
len(all_files_source2)

15944

In [94]:
len(all_json_source1)

740

In [95]:
len(all_json_source2)

7972

In [110]:
def extract_titles(filenamelist,source,basedir=BASE_DIR):
    page_titles = []
    for filename in filenamelist:
        filepath = os.path.join(BASE_DIR,source,filename)
        with open(filepath) as json_file:
            json_ob = json.load(json_file)
            page_title = json_ob['<page title>']
            page_titles.append(page_title)
    return page_titles                

In [121]:
def createDistributedRapresentation(recordTitleList):
    embeddingMap = {}
    for recordtitle in recordTitleList:
        currentTitle = recordtitle
        tokens_title = tokenizer.tokenize(currentTitle)
        filtered_title = [word for word in tokens_title if word not in stop_words]
        #processo di creazione del vettore di embedding
        numeratoreVec = np.zeros(300)
        ntokens = 0
        for token in filtered_title:
            ntokens += 1
            embeddingVector = wordToEmbeddingMap.get(token)
            if embeddingVector is not None:
                numeratoreVec += embeddingVector
        tupleEmbedding = numeratoreVec /ntokens
        embeddingMap[currentTitle]= tupleEmbedding
    return embeddingMap

In [111]:
titlesList_source1 = extract_titles(all_json_source1,source1)

In [112]:
titlesList_source2 = extract_titles(all_json_source2,source2)

In [122]:
embedding_source1 = createDistributedRapresentation(titlesList_source1)
embedding_source2 = createDistributedRapresentation(titlesList_source2)

In [129]:
from sklearn.metrics.pairwise import cosine_similarity

embedding55 = list(embedding_source1.values())[0].reshape(1,-1)
embedding273 = list(embedding_source2.values())[100].reshape(1,-1)

cosineSim = cosine_similarity(embedding55,embedding273)
cosineSim

array([[0.44022907]])